In [43]:
# Libraries for developing a Neural Network
import tensorflow as tf 
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.utils import to_categorical 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import math
from sklearn.preprocessing import StandardScaler
from keras.losses import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
sns.set_style('darkgrid')
plt.rcParams['figure.dpi'] = 108

from collections import defaultdict
import random
from random import randrange
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score


from sklearn.metrics import roc_auc_score

In [44]:
dt = pd.read_csv("model_data/data.csv",sep ="\t")

dt['Real']=dt.apply(lambda x: x.Real if x.Real ==1 else 0,axis =1)
y = dt.Real
test_dt =pd.read_csv("model_data/test_data.csv",sep ="\t")
ds = dt.drop('Real',axis =1)

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(ds, y, test_size=0.2, random_state=1)
print("Training set has {} instances. Test set has {} instances.".format(X_train.shape[0], X_test.shape[0]))

Training set has 6364 instances. Test set has 1591 instances.


In [46]:
# poly =PolynomialFeatures(2,include_bias = False)
# poly.fit(X_train)
# X_train= poly.transform(X_train)
# X_test= poly.transform(X_test)
# test_dt = poly.transform(test_dt)

In [47]:
# standard
scaler = StandardScaler()
scaler.fit(X_train)
X_train= scaler.transform(X_train)
X_test= scaler.transform(X_test)
#dt = scaler.transform(dt)
test_dt = scaler.transform(test_dt)
#dt['Real'] =dt.apply(lambda x: 0 if x.Real ==-1 else 1, axis =1)


In [18]:
if __name__ == "__main__":
    #定义神经网络的参数
    w = tf.Variable(tf.random_normal([,5],stddev=1,seed=1))
    b = tf.Variable(tf.random_normal([1],stddev=1,seed=1))
    #定义输入和输出
    x = tf.placeholder(tf.float32,shape=(None,2),name="x-input")
    y_ = tf.placeholder(tf.float32,shape=(None,1),name="y-input")
    #定义神经网络的前向传播过程
    y = tf.nn.sigmoid(tf.matmul(x,w) + b)
    #基于min和max对张量t进行截断操作，为了应对梯度爆发或者梯度消失的情况
    cross_entropy = -tf.reduce_mean(y_ * tf.log(tf.clip_by_value(y,1e-10,1.0))+(1-y_) * tf.log(tf.clip_by_value(1-y,1e-10,1.0)))
    # 使用Adadelta算法作为优化函数，来保证预测值与实际值之间交叉熵最小
    train_step = tf.train.AdamOptimizer(0.01).minimize(cross_entropy)
    #通过随机函数生成一个模拟数据集
    rdm = RandomState(1)
    # 定义数据集的大小
    dataset_size = 100
    # 模拟输入是一个二维数组
    X = rdm.rand(dataset_size,2)
    #定义输出值，将x1+x2 < 1的输入数据定义为正样本
    Y = [[int(x1+x2 < 1)] for (x1,x2) in X]
    #创建会话运行TensorFlow程序
    with tf.Session() as sess:
        #初始化变量  tf.initialize_all_variables()
        init = tf.initialize_all_variables()
        sess.run(init)
        #设置神经网络的迭代次数
        steps = 500
        for i in range(steps):
            #通过选取样本训练神经网络并更新参数
            for (input_x,input_y) in zip(X,Y):
                input_x = np.reshape(input_x,(1,2))
                input_y= np.reshape(input_y,(1,1))
                sess.run(train_step,feed_dict={x:input_x,y_:input_y})
            #每迭代1000次输出一次日志信息
            if i % 100 == 0:
                # 计算所有数据的交叉熵
                total_cross_entropy = sess.run(cross_entropy,feed_dict={x:X,y_:Y})
                # 输出交叉熵之和
                print("After %d training step(s),cross entropy on all data is %g"%(i,total_cross_entropy))
        #预测输入X的类别
        pred_Y = sess.run(y,feed_dict={x:X})
        index = 1
        for pred,real in zip(pred_Y,Y):
            print(pred,real)

SyntaxError: invalid syntax (<ipython-input-18-787f317c7bd5>, line 3)

In [48]:
def auroc(y_true, y_pred):
    return tf.py_function(roc_auc_score, (y_true, y_pred), tf.double)


In [50]:
# Model arguments
args = dict(x = X_train,
            y = y_train,
            epochs=100,
            validation_split=0.2,
            verbose=2,
            shuffle=True)

# Layer definition
input_layer = Input(shape=(5,))
hidden_layer_1 = Dense(100, activation='relu',activity_regularizer=regularizers.l1(10e-5))(input_layer)

#hidden_layer_1 = Dropout(0.3)(hidden_layer_1)
#hidden_layer_2 = Dense(8, activation='sigmoid')(hidden_layer_1)
hidden_layer_2 = Dense(50, activation='relu')(hidden_layer_1)
hidden_layer_2 = Dropout(0.3)(hidden_layer_2)
output_layer = Dense(2, activation='softmax')(hidden_layer_2)
model = Model(inputs=input_layer, outputs=output_layer)

# Model set up
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy', auroc])
model.compile(tf.keras.optimizers.RMSprop(learning_rate=0.01),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()
#model.compile(loss='categorical_crossentropy', optimizer='adam',metrics=['accuracy', auroc])

Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_10 (InputLayer)        [(None, 5)]               0         
_________________________________________________________________
dense_27 (Dense)             (None, 100)               600       
_________________________________________________________________
dense_28 (Dense)             (None, 50)                5050      
_________________________________________________________________
dropout_6 (Dropout)          (None, 50)                0         
_________________________________________________________________
dense_29 (Dense)             (None, 2)                 102       
Total params: 5,752
Trainable params: 5,752
Non-trainable params: 0
_________________________________________________________________
Epoch 1/100
160/160 - 0s - loss: 0.5417 - accuracy: 0.7325 - val_loss: 0.5785 - val_accuracy: 0.6441
Epoch 2/100
160/160 - 

Epoch 72/100
160/160 - 0s - loss: 0.2568 - accuracy: 0.9083 - val_loss: 0.1713 - val_accuracy: 0.9505
Epoch 73/100
160/160 - 0s - loss: 0.2482 - accuracy: 0.9083 - val_loss: 0.3332 - val_accuracy: 0.8657
Epoch 74/100
160/160 - 0s - loss: 0.2437 - accuracy: 0.9149 - val_loss: 1.3184 - val_accuracy: 0.7761
Epoch 75/100
160/160 - 0s - loss: 0.2454 - accuracy: 0.9203 - val_loss: 1.6462 - val_accuracy: 0.7400
Epoch 76/100
160/160 - 0s - loss: 0.2337 - accuracy: 0.9173 - val_loss: 0.1314 - val_accuracy: 0.9678
Epoch 77/100
160/160 - 0s - loss: 0.2294 - accuracy: 0.9191 - val_loss: 1.4693 - val_accuracy: 0.5734
Epoch 78/100
160/160 - 0s - loss: 0.2349 - accuracy: 0.9142 - val_loss: 0.1952 - val_accuracy: 0.9191
Epoch 79/100
160/160 - 0s - loss: 0.2416 - accuracy: 0.9220 - val_loss: 0.1464 - val_accuracy: 0.9647
Epoch 80/100
160/160 - 0s - loss: 0.2266 - accuracy: 0.9206 - val_loss: 0.1723 - val_accuracy: 0.9466
Epoch 81/100
160/160 - 0s - loss: 0.2222 - accuracy: 0.9193 - val_loss: 0.1193 - v

In [ ]:
# Execute training
model.fit(**args)



In [ ]:


# Prediction for test
predict = model.predict(X_test)
classes = []
classes = predict.argmax(axis=-1)

In [ ]:
print('Accuracy: ' + "{:.4f}".format(accuracy_score(y_test, classes)))
print(classes)

In [ ]:
print(predict)

In [ ]:
auc = roc_auc_score(y_test, classes)
print(auc)


In [ ]:
predict_test = model.predict(test_dt)
print(predict_test)

In [ ]:
print(np.argmax(predict_test, axis=1))

In [ ]:
table1 = numpy.array(predict_test)
test_result = list(table1(:,1)) 

Accuracy: 0.9679
[1 0 1 ... 1 0 1]
[[1.5004945e-25 1.0000000e+00]
 [9.9997842e-01 2.1595813e-05]
 [2.1287208e-02 9.7871280e-01]
 ...
 [5.6260444e-02 9.4373953e-01]
 [9.6354610e-01 3.6453933e-02]
 [2.1140231e-02 9.7885978e-01]]
0.9680016560948399


In [52]:
predict_test = model.predict(test_dt)
print(predict_test)

[[0.97394794 0.02605201]
 [0.98152    0.01847997]
 [0.20683786 0.7931621 ]
 ...
 [0.997317   0.002683  ]
 [0.9963805  0.00361954]
 [0.9972898  0.00271026]]


In [53]:
print(np.argmax(predict_test, axis=1))

[0 0 1 ... 0 0 0]


In [55]:
table1 = numpy.array(predict_test)
test_result = list(table1(:,1)) 

SyntaxError: invalid syntax (<ipython-input-55-bf83eb8bb038>, line 2)